![title](http://file.scirp.org/Html/6-9101686/f799e10c-50bd-48ec-9344-49d767083be5.jpg)

In the last notebook, we saw how to build predictive models using decision trees. However, we also learned that these trees are susceptible to variance and unbalanced data. How do we solve this?

We can combine several trees, get the prediction for each of them. Then, we run a majority vote between all the trees to call the final prediction. If we just take a sample of the data each time,  then we can create multiple, different trees. **However**, these trees may just come to the same final conclusion as the original single tree, because of the general strength of some of the features. So one other thing we need to do is limit the number of features that we use on the tree. 

This approach to generating various decision trees with random samples and random features is called **Random Forest**. A random forest will give us a more robust model for prediction, that is less susceptible to overfitting.

## Random Forest

Let's see it in action with the HR dataset again, and with the decision trees we built in the previous notebook. I created a class that implements them already, so no need to write them again, just import them.

In [85]:
from DecisionTree import ClassificationTree
from DecisionTree import RegressionTree
import pandas as pd
import numpy as np

In [86]:
HR = pd.read_csv("HR_comma_sep.csv")
HR.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,dept,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


Let's do the classification task with a Random Forest this time around. To do this, we just have to resample the dataset columns and rows for each tree. This can be done with random sampling in pandas easily. However, we still have to balance the dataset.

In [87]:
import random
def Balance(data,target = "left"):
    counts = data[target].value_counts()
    most_common = counts.index[0]
    common_sample = data.loc[data[target] == most_common]
    random_sample  = np.random.choice(len(common_sample), replace=False, size=counts[1])
    final_sample = common_sample.iloc[random_sample]
    balanced = final_sample.append(data.loc[data[target] == counts.index[1]])
    return balanced

def Random_Forest(data,n_trees = 10,target="left",sample_size = 0.8,max_features = None,max_depth = 3):
    if not max_features:
        max_features = int(pow(data.shape[1],0.5))
    
    trees = []
    balanced_sample = Balance(data)
    print("Max features for random trees: " + str(max_features))
    for i in range(n_trees):
        #Create the row random sample.

        row_sample = balanced_sample.sample(frac = sample_size)
        #Create the column random sample, without the prediction variable, then readd it so it isn't dropped.
        temp_target = row_sample[target]
        num_features = random.randint(1, max_features)
        column_sample = row_sample.drop(target,axis =1).sample(n = num_features, axis = 1)
        column_sample[target] = temp_target
        tree = ClassificationTree(X = column_sample,target = target,max_depth = max_depth)
        tree.Build_Tree()
        print("Tree number " + str(i + 1) + ". Number of features: " + str(num_features))
        tree.Print_Tree(tree.root)
        trees.append(tree)
    return trees

Forest = Random_Forest(HR, max_depth = 4)

Max features for random trees: 3
Tree number 1. Number of features: 3
  0 : Work_accident = 0
   1 : average_montly_hours <= 253.0
    2 : salary = low
     3 : 0
     3 : 0
    2 : salary = low
     3 : 1
     3 : 1
   1 : average_montly_hours <= 203.0
    2 : salary = low
     3 : 0
     3 : 0
    2 : salary = low
     3 : 0
     3 : 0
Tree number 2. Number of features: 2
  0 : number_project <= 2.0
   1 : promotion_last_5years = 0
    2 : 1
    2 : 1
   1 : promotion_last_5years = 0
    2 : 0
    2 : 0
Tree number 3. Number of features: 3
  0 : promotion_last_5years = 0
   1 : salary = low
    2 : dept = sales
     3 : 1
     3 : 1
    2 : dept = sales
     3 : 0
     3 : 0
   1 : salary = medium
    2 : dept = sales
     3 : 0
     3 : 0
    2 : dept = management
     3 : 0
     3 : 0
Tree number 4. Number of features: 3
  0 : Work_accident = 0
   1 : time_spend_company <= 4.0
    2 : dept = sales
     3 : 0
     3 : 0
    2 : dept = technical
     3 : 1
     3 : 1
   1 : dept = ma

There are other things that can be used on the creation of a random forest, like changing the way the prediction is chosen (using logistic regression or another method), or giving the max depth more priority than the max features. But I'm limiting it to this as to leave the rest to you.  

The next thing to do is predicting. To predict, we run the row to predict over each tree in the random forest. Each final prediction is stored in a list, and then the most common prediction will be selected as the final one.

In [94]:
def Forest_Predict(row,forest,verbose = False):
    predictions = []
    for tree in forest:
        predictions.append(tree.Predict(tree.root,row))
        
    return max(set(predictions), key=predictions.count)
    

sample = HR.sample(frac = 0.2, random_state = 420)
predictions = []
for index,row in sample.iterrows():

    prediction = Forest_Predict(row,Forest)
    print("Expected " + str(row.left) + " Got " + str(prediction) )
    predictions.append(prediction == row.left)
    
accuracy = sum(predictions)/ len(sample)
print("Accuracy was {} %".format(accuracy * 100))

Expected 0 Got 0
Expected 0 Got 0
Expected 0 Got 0
Expected 0 Got 0
Expected 0 Got 0
Expected 1 Got 0
Expected 0 Got 0
Expected 0 Got 0
Expected 0 Got 0
Expected 1 Got 0
Expected 1 Got 0
Expected 0 Got 0
Expected 0 Got 0
Expected 1 Got 1
Expected 0 Got 0
Expected 0 Got 0
Expected 0 Got 1
Expected 0 Got 0
Expected 1 Got 1
Expected 0 Got 0
Expected 0 Got 0
Expected 0 Got 0
Expected 1 Got 0
Expected 0 Got 0
Expected 1 Got 0
Expected 0 Got 0
Expected 0 Got 0
Expected 0 Got 0
Expected 0 Got 0
Expected 1 Got 0
Expected 1 Got 1
Expected 0 Got 0
Expected 0 Got 0
Expected 0 Got 1
Expected 0 Got 0
Expected 0 Got 0
Expected 0 Got 0
Expected 0 Got 0
Expected 0 Got 0
Expected 0 Got 0
Expected 0 Got 0
Expected 0 Got 0
Expected 0 Got 0
Expected 0 Got 0
Expected 1 Got 0
Expected 0 Got 1
Expected 0 Got 0
Expected 0 Got 0
Expected 0 Got 0
Expected 1 Got 0
Expected 0 Got 0
Expected 1 Got 0
Expected 0 Got 0
Expected 1 Got 0
Expected 0 Got 0
Expected 0 Got 0
Expected 0 Got 0
Expected 0 Got 0
Expected 0 Got

And that's it! For regression problems, instead of taking the majority vote, we take the mean of the tree predictions, which in turn are the means of the splits. Random forests provide a more robust prediction model, by taking the results of several, weaker predictions. This is called model **ensembling**. Random forests are one of the models that implement this concept using decision trees. Another one is **Gradient Boosting**. 

## Gradient Boosted Trees

On gradient boosting , we apply the same concept that we have learned about gradient descent. We update the model based on the errors of the predictions. However, for gradient boosting this works a little differently. Here's the step by step:

1. Fit the model to a base prediction $F_1(x) = y$
2. Fit the model again to the error of the prediction. $H_1(x) = y - F_1(x)$ (Hypothesis Function)
3. Update the prediction by adding the base prediction and the hypothesis multiplied by a step size (learning rate). 
    $F_2(x) = F_1(x) + \alpha * H_1(x)$
4. Repeat steps 2 to 3 for N boosting rounds.

What this process does is diminish the error on each round. This is done on step two, where a weak learning model (linear regression, small decision trees) are tasked with predicting the residual or error of the previous prediction, instead of the actual values of the predictions themselves. After this, the predicted errors are added to the base prediction, and become the new base prediction for the next boosting round. In the case of decision trees, the actual splits of the trees may change during boosting rounds as well. When boosting rounds are finished, we fit the tree a final time on the final base prediction. Let's see it in action:

In [89]:
def Gradient_Boosting(data,target,n_rounds=10,lrate = 0.9,early_stop = 1e-8,max_depth = 3):
    temp = data.copy()
    #Get the base prediction.
    base_pred = temp[target].median()
    base_pred = np.repeat(base_pred,len(temp))
    tree = None

    #Train for N training rounds.
    for i in range(n_rounds):
        residual = temp[target] - base_pred
        if abs(residual.mean()) < early_stop:
            print("Early stopping at round {}, model has finished training".format(i))
            break
            
        temp["residual"] = residual 
        tree = RegressionTree(X = temp.drop(target, axis = 1),target = "residual", max_depth = max_depth)
        tree.Build_Tree()
        #This like takes a long time since it's going row by row. It could be optimized by changing the underlying tree function.
        predictions = np.asarray([tree.Predict(tree.root,row) for index,row in temp.iterrows()])
        
        base_pred = base_pred + (lrate * predictions)
        
    
    temp.drop("residual", axis = 1,inplace = True)
    temp["prediction"] = base_pred
    tree = RegressionTree(X = temp.drop(target,axis = 1), target = "prediction")
    tree.Build_Tree()
    return tree
    
    
    

boosted_tree = Gradient_Boosting(HR,"satisfaction_level")

Early stopping at round 7, model has finished training


In practice, the algorithm would be much faster, and the learning rate would be different for each leaf (final split) in the tree.Let's see the resulting tree ( feel free to compare it to the one in the previous notebook).

In [90]:
boosted_tree.Print_Tree(boosted_tree.root)

  0 : left = 1
   1 : time_spend_company <= 3.0
    2 : number_project <= 2.0
     3 : 0.4067611327354653
     3 : 0.32475570237438145
    2 : number_project <= 5.0
     3 : 0.7689533546746147
     3 : 0.12315123332345912
   1 : time_spend_company <= 3.0
    2 : average_montly_hours <= 163.0
     3 : 0.6918221832695975
     3 : 0.7028469183108452
    2 : number_project <= 5.0
     3 : 0.6205426107481328
     3 : 0.2881373432245035


And finally, let's test the model with a random test sample.

In [91]:
def Reg_Test(sample,tree,target = "satisfaction_level", verbose = True):
    error = 0
    for index,row in sample.iterrows():
        real = row[target]
        prediction = tree.Predict(tree.root,row)
        if verbose:
            print("Expected: " + str(real) + " Predicted: " + str(prediction))
        error += (real - prediction) ** 2
    MSE  = error/len(sample)
    RMSE = pow(MSE,0.5)
    return RMSE
    
              
sample = HR.sample(frac = 0.2, random_state = 777)
Reg_Test(sample,boosted_tree,verbose = False)

0.19090684900659596

There are a lot of variants to the gradient boosting algorithm, and it's still one of the most widely used to this day with several implementations used in industry that we will see later. It's not limited to trees either, but it's mostly used with trees. Some implementations are really complex, but at base it's still the same process for this trusty model.

## SKLearn

Let's see the sklearn implementation for both Random Forest and Gradient Boosting on the same problems.

In [101]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

#Label encode salary.
HRCopy = HR.copy()

lb = LabelEncoder()
HRCopy.salary = lb.fit_transform(HRCopy.salary)

#One Hot Encode department
OHdept = pd.get_dummies(HRCopy.dept)
HRCopy.drop("dept", axis = 1,inplace = True)
HRCopy[OHdept.columns] = OHdept


Xtrain,Xtest,ytrain,ytest = train_test_split(HRCopy.drop("left",axis = 1),HRCopy.left,test_size = 0.2,random_state = 420)

rf = RandomForestClassifier()
rf.fit(Xtrain,ytrain)
rf.score(Xtest,ytest)

0.99233333333333329

A much better accuracy than our handcrafted model. The crafted one could be improved. 

In [105]:
from sklearn.ensemble import GradientBoostingRegressor

Xtrain,Xtest,ytrain,ytest = train_test_split(HRCopy.drop("satisfaction_level",axis = 1),
                                             HRCopy.satisfaction_level,test_size = 0.2,random_state = 420)

gb = GradientBoostingRegressor()
gb.fit(Xtrain,ytrain)
pred = gb.predict(Xtest)

RMSE = np.sqrt(((ytest - pred)**2).mean())
RMSE

0.18217357255176933

Again, a better score than our model. However, the point of the implementations is to get the intuition behind them in a more granular fashion. With this, we've seen the power of combining individual predictors like trees into a more robust, more resistant model.